In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/dbfs/FileStore/tables/My_First_Project_fe78171bdba0-bdf8d.json'
import pandas as pd
from google.cloud import bigquery
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from google.cloud import bigquery
client = bigquery.Client()
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
    '/dbfs/FileStore/tables/My_First_Project_fe78171bdba0-bdf8d.json',
)

import pandas_gbq

## Import Jobs Dataframe from Google BigQuery

In [3]:
project_id='deep-rigging-245523'
query = "SELECT * FROM Projects.job_listings"
jobs_df = (pandas_gbq.read_gbq(query,project_id=project_id, dialect='standard', credentials=credentials)
           .drop_duplicates(subset = "description"))
print(len(jobs_df))
jobs_df.head()

523
 Out[ 4 ]: 
 title company \
0 Staff Data Scientist Valassis Digital 
1 People Analytics - data scientist IBM 
2 Data Scientist Analyst Domino's 
3 Deloitte Global- Data Scientist Deloitte 
4 Data Scientist (Anti-Fraud Team) Walmart eCommerce 

 location \
0 Morrisville, NC 
1 Armonk, NY 
2 Ann Arbor, MI 
3 Glen Mills, PA 
4 San Francisco Bay Area, CA 

 summary \
0 Staff Data Scientist – Machine Lea... 
1 Formulating mathematical approache... 
2 Proficient in SQL and R or Python.... 
3 Train other data scientists in are... 
4 Deep knowledge of statistical anal... 

 link \
0 https://www.indeed.com/pagead/clk?mo=r&ad=-6NY... 
1 https://www.indeed.com/rc/clk?jk=f423a0f019acd... 
2 https://www.indeed.com/rc/clk?jk=e6f0a2235caab... 
3 https://www.indeed.com/rc/clk?jk=2f3637eb11fb2... 
4 https://www.indeed.com/company/Walmart-eCommer... 

 description 
0 Staff Data Scientist – Machine Learning & Real... 
1 Introduction At IBM, work is more than a job -... 
2 Domino’s, which began in 1960 as a single stor... 
3 Deloitte leads with purpose, solving complex i... 
4 Imagine working in an environment where one ex...

## Import Skills Dataframe from Google BigQuery

In [5]:
query = "SELECT * FROM Projects.skills"
skills_df = pandas_gbq.read_gbq(query,project_id=project_id, dialect='standard', credentials=credentials)
skills_df = skills_df.rename(columns={'string_field_0': 'name'})
print(len(skills_df))
skills_df.head(n=10)

36944
 Out[ 6 ]: 
 name
0 (ISC)2
1 .NET
2 .NET CLR
3 .NET Compact Framework
4 .NET Framework
5 .NET Remoting
6 .Net Core
7 .com
8 .htaccess
9 1-4 Units

## User Inputs

In [7]:
# input_profession = input("Enter your profession, or enter x to use default inputs ")
# if input_profession == "x":
input_profession = "Data Analyst"
input_skills = ["Python", "R", "SQL", "Tableau", "Power BI", "Salesforce", "Excel",
                         "Machine Learning", "Statistics", "Analytics", "Visualization", "Web Scraping"]
print("Default profession: ", input_profession)
print("Default skills: ", input_skills)
# else: 
#     amount_skills = int(input("How many skills do you have? "))
#     input_skills = pd.Series()
#     for i in range(amount_skills):
#         temp_skill = input("Name skill #" + str(i + 1) + " ")
#         input_skills = input_skills.append(pd.Series(temp_skill))

Default profession: Data Analyst
Default skills: ['Python', 'R', 'SQL', 'Tableau', 'Power BI', 'Salesforce', 'Excel', 'Machine Learning', 'Statistics', 'Analytics', 'Visualization', 'Web Scraping']

In [8]:
input_skills_string = " ".join(input_skills) + " " + input_profession

In [9]:
# Import TfidfVectorizer
jobs_df["words"] = jobs_df["title"] + jobs_df["summary"] + jobs_df["description"]

# Create a TfidfVectorizer: tfidf
tfidf = TfidfVectorizer(stop_words="english", 
                        analyzer='word',
                        vocabulary = map(lambda x: str(x).lower(), skills_df.name.values),
                        ngram_range=(1, 3), max_df=0.1, min_df=0.0025, max_features=None)

# Apply fit_transform to job descriptions: tfidf_jobs
job_summaries = jobs_df["words"].astype(str).copy()
job_summaries = job_summaries.append(pd.Series([input_skills_string]))  

tfidf_result = tfidf.fit_transform(job_summaries.values)

tfidf_jobs = pd.DataFrame(data = tfidf_result.toarray(), 
                          index = job_summaries, 
                          columns = tfidf.get_feature_names())

tfidf_jobs.tail()

Out[ 9 ]: 
 (isc)2 .net .net clr \
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 0.0 0.0 
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 0.0 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 0.0 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 0.0 0.0 
Python R SQL Tableau Power BI Salesforce Excel ... 0.0 0.0 0.0 

 .net compact framework \
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 
Python R SQL Tableau Power BI Salesforce Excel ... 0.0 

 .net framework \
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 
Python R SQL Tableau Power BI Salesforce Excel ... 0.0 

 .net remoting .net core \
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 0.0 
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 0.0 
Python R SQL Tableau Power BI Salesforce Excel ... 0.0 0.0 

 .com .htaccess \
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 0.0 
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 0.0 
Python R SQL Tableau Power BI Salesforce Excel ... 0.0 0.0 

 1-4 units ... zumba \
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 ... 0.0 
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 ... 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 ... 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 ... 0.0 
Python R SQL Tableau Power BI Salesforce Excel ... 0.0 ... 0.0 

 zumba instruction zuora \
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 0.0 
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 0.0 
Python R SQL Tableau Power BI Salesforce Excel ... 0.0 0.0 

 zymography zynx zyxel \
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 0.0 0.0 
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 0.0 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 0.0 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 0.0 0.0 
Python R SQL Tableau Power BI Salesforce Excel ... 0.0 0.0 0.0 

 z/os z/vm zlinux \
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 0.0 0.0 
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 0.0 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 0.0 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 0.0 0.0 
Python R SQL Tableau Power BI Salesforce Excel ... 0.0 0.0 0.0 

 zseries 
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 
FGP Sec I-Intake/Sched (37.5) - Cantonese/Manda... 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 
Full Time Opportunities for PhD Students or Rec... 0.0 
Python R SQL Tableau Power BI Salesforce Excel ... 0.0 

[5 rows x 36944 columns]

In [10]:
def get_recommendations(features, index):

    #cosine similarity
    norm_features = normalize(features.toarray())
    jobs_rec_df = pd.DataFrame(norm_features, index = index, columns = tfidf.get_feature_names())
    current_item = jobs_rec_df.loc[input_skills_string].values.reshape(-1,1)
    similarities = jobs_rec_df.dot(current_item).reset_index()
    recommended_jobs = similarities.nlargest(n=6, columns=0)
    recommended_jobs = pd.merge(recommended_jobs.reset_index(), 
                                jobs_df, 
                                left_on="index", 
                                right_on="words")[["title","company","summary","link"]]
    return jobs_rec_df, similarities, recommended_jobs

In [11]:
jobs_rec_df, similarities, recommended_jobs = get_recommendations(tfidf_result, job_summaries)

In [12]:
similarities.sort_values(0, ascending = False).head()

Out[ 12 ]: 
 index 0
523 Python R SQL Tableau Power BI Salesforce Excel... 1.000000
395 Data Scientist Projects will includ... 0.255816
495 Junior Data Scientist PMC is buildi... 0.233016
402 Data Scientist Medscape, a division... 0.219162
399 Data Scientist Reframe objectives a... 0.214770

In [13]:
for i in range(5):
    print(recommended_jobs["title"][i], "at", (recommended_jobs["company"][i]))
    print(recommended_jobs["summary"][i])
    print(recommended_jobs["link"][i])
    print()
    print()

Data Scientist at Hire Tech Solutions
 Projects will include creating machine learning microservices, detecting patterns and anomalies, analysis and interpretation of data, and data visualization....
https://www.indeed.com/company/Hire-Tech-Solutions/jobs/Data-Scientist-466f6ae6ab72506d?fccid=821241ecf9db90ef&vjs=3


Junior Data Scientist at Protect My Car
 PMC is building out a small team of talented professionals with demonstrated ability to conduct rigorous statistical analysis and apply machine learning...
https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Cc2767xsnTKh3D1ndxis86rSe8bm3rZJt-h0l22W5JsqVSPirPG7nXDv4eNXyw1s-UDmSqiHCKjDOqRXiD1rrvXb4iCZWBN71e31Fw4mZ3DhiDh6cuu3P8OlrrAbBcCk43ZmgQqEUjLSkWyaQeoONv6A02UjczxoGdAK7Cy2znFbM207UUMdF-PM2-61lKHybnVtw1UNU59-d3ylwFHoI_u1DG2mPMLCsraHoTj-kEac5S5_LLjdaRV8bRP4bW-N9sdj1SyQNRO0HfP2-FXLQQ8IOEhqGADC3Pwivr4iLfS93XVamqethjih-W1TxftLvn2lKue2HhrNIxH_WHGnxicSmSYz161pvSu9-P0pZo6UH_9FzesjusptNat7u2SEzf6JMumTpxXGMsa_tYwQEaQDXX5yKpxepirHQsSPpmGWhRcNSK2pXp&p=3&fvj=1&vjs=3


Data Scientist at WebMD
 Medscape, a division of WebMD, develops and hosts physician portals and related mobile applications that make it easier for physicians and healthcare...
https://www.indeed.com/rc/clk?jk=6b8c0d8cfec7443c&fccid=2f7cb114b730b276&vjs=3


Data Scientist at Sony Music Entertainment
 Reframe objectives as machine learning tasks that can deliver actionable insights, accurate predictions, and effective optimization....
https://www.indeed.com/rc/clk?jk=c496f791737c04c4&fccid=c5284eeef83d4d1c&vjs=3


Junior Data Scientist at BerlandTeam
 Advance our capabilities for how statistics and machine learning should be used in analyzing digital trends, CRM data and quantitative research....
https://www.indeed.com/rc/clk?jk=af6e9e4d7d155600&fccid=37d7d967e20c750c&vjs=3

In [14]:
skills_rec = cosine_similarity(jobs_rec_df.transpose(), jobs_rec_df.transpose())
skills_rec_df = pd.DataFrame(skills_rec, index = list(skills_df.name.values), columns = list(skills_df.name.values))
display(skills_rec_df.head())

Out[ 15 ]: 
 (ISC)2 .NET .NET CLR .NET Compact Framework \
(ISC)2 0.0 0.0 0.0 0.0 
.NET 0.0 0.0 0.0 0.0 
.NET CLR 0.0 0.0 0.0 0.0 
.NET Compact Framework 0.0 0.0 0.0 0.0 
.NET Framework 0.0 0.0 0.0 0.0 

 .NET Framework .NET Remoting .Net Core .com \
(ISC)2 0.0 0.0 0.0 0.0 
.NET 0.0 0.0 0.0 0.0 
.NET CLR 0.0 0.0 0.0 0.0 
.NET Compact Framework 0.0 0.0 0.0 0.0 
.NET Framework 0.0 0.0 0.0 0.0 

 .htaccess 1-4 Units ... Zumba \
(ISC)2 0.0 0.0 ... 0.0 
.NET 0.0 0.0 ... 0.0 
.NET CLR 0.0 0.0 ... 0.0 
.NET Compact Framework 0.0 0.0 ... 0.0 
.NET Framework 0.0 0.0 ... 0.0 

 Zumba Instruction Zuora Zymography Zynx Zyxel \
(ISC)2 0.0 0.0 0.0 0.0 0.0 
.NET 0.0 0.0 0.0 0.0 0.0 
.NET CLR 0.0 0.0 0.0 0.0 0.0 
.NET Compact Framework 0.0 0.0 0.0 0.0 0.0 
.NET Framework 0.0 0.0 0.0 0.0 0.0 

 Z/OS Z/VM ZLinux ZSeries 
(ISC)2 0.0 0.0 0.0 0.0 
.NET 0.0 0.0 0.0 0.0 
.NET CLR 0.0 0.0 0.0 0.0 
.NET Compact Framework 0.0 0.0 0.0 0.0 
.NET Framework 0.0 0.0 0.0 0.0 

[5 rows x 36944 columns]

In [15]:
display(skills_rec_df.head())

(ISC)2,.NET,.NET CLR,.NET Compact Framework,.NET Framework,.NET Remoting,.Net Core,.com,.htaccess,1-4 Units,1-Wire,1-to-1 Marketing,100% Financing,100-105,100D,1031 Exchanges,1099 Preparation,10G Ethernet,10K,10Q,10b5-1 Plans,10gR2,10gen,1120S,11781,11gR1,11gR2,123D Catch,123D Design,12c,12c Release 2,144A,16PF,1709,18th Century,1940 Act,19th Century,1H NMR,1Password,1SYNC,1ShoppingCart,2-D,2-D Design,20-20 CAP Studio,20/20 Design,20/20 Design Software,20/20 Technologies,200-101,2008 R2,2012 R2,2013 (Lync),2014a,2015a,2016 R2,2017a,203K,20th Century,21 CFR,21st Century Skills,220-801,220-802,220-901,220-902,24 Hour Emergency Service,24x7,24x7 Production Support,27.x,2D Animation,2D CAD Drawing,2D Drawing,2D Software,2D art,2D graphics,2D to 3D Conversion,2Do,2G,2V0,2nd Line,2v0-641,2ε,340B,35mm,360,360 Assessments,360 Campaigns,360 Degree Assessment,360 Feedback,360 Recruitment,360s,365,37signals,3CX,3Com NBX,3Com Switches,3D + Animation,3D Animation,3D Architectural Rendering,3D Coat,3D Design,3D Drawing,3D GIS,3D Graphic Design,3D Graphics,3D Home Architect,3D Mapping,3D Materials,3D Math,3D Modeling,3D Modeling Software,3D Packaging,3D Particles and Dynamics,3D Printing,3D Production,3D Prototyping,3D Reconstruction,3D Rendering,3D Scanning,3D Secure,3D Seismic Interpretation,3D Simulation,3D Studio Max,3D Studio Viz,3D Textures,3D Tracking,3D Typography,3D Visualization,3D and Animation,3D displays,"3D, Animation & CAO","3D, Animation und CAD","3D, animación y CAD",3D-Beleuchtung,3D-Druck,3D-Grafik,3D-Materialien,3D-Modellierung,3D-Partikel und Dynamik,3D-Texturen,3DCom,3DES,3DEqualizer,3DTV,3DVIA Composer,3Delight,3G,3GPP,3GPP2,3P,3PAR,3d,3ds Max,3rd Line Support,3rd Party Integrations,3rd Party Liaison,3rd Party Partnerships,3rd Party Relationships,3rd Party Software Integration,4 Disciplines of Execution,401(k) Retirement Savings Plans,401k Rollovers,403 b,404 Compliance,409A,40G,457 Plans,480interactive,49CFR,4G,4GL,4K,4PL,4th Dimension,4x4,5 Why,5.1 Mixing,501c3,504 Plans,529 Plans,55+ Communities,5500s,5D,5ESS,5S,5Y,60D,63 Licenses,6502 Assembly,650D,68HC11,68K,68k Assembly,6D,6S,7 QC Tools,700D,701,7D Mark II,8 Wastes,8-D,802.11a,802.11b,802.11g,802.11i,802.11n,802.15.4,802.16e,802.1ag,802.1d,802.1q,802.1x,8051 Assembly,8051 Microcontroller,80x86,837I,837P,89C51,8D Problem Solving,960 Grid System,@Task,Eclairage 3D,Eclairage et rendu,Eclairage photo,Equipement photo,Equipement video,A + Certified,A&D,A&E,A&H,A&P,A&R Administration,A&U,A+,A++,A-133,A-GPS,A-frames,A/B Testing,A/R Collections,A/R Management,A/R analysis,A1 Assessor,A10,A2,A2A,A3,A3 Thinking,A320,A330,AAAHC,AACR2,AAMS,AAP,AAR,AASHTO,AAUS Scientific Diver,ABAP Web Dynpro,ABAP-OO,ABB 800xA,ABBYY,ABBYY FineReader,ABC Analysis,ABC Flowcharter,ABCP,ABEL,ABI,ABLS,ABO Certified,ABSYNTH,ABV,AC Drives,AC Nielsen,AC/DC,ACAD,ACARS,ACATS,ACBS,ACD,ACD Management,ACDSee,ACE,ACE Certified,ACE Certified Personal Trainer,ACF2,ACFE,ACH,ACI Codes,ACIS,ACLS Instruction,ACMS,ACOA,ACORD,ACP,ACPI,ACR,ACSA,ACSC,ACSLS,ACSM,ACSM Health Fitness,ACSP,ACSR,ACSS,ACT CRM,ACT prep,ACT!,AD Migration,ADA Compliance,ADA guidelines,ADABAS,ADB,ADB Adapter,ADCs,ADDIE,ADDM,ADE,ADEA,ADFS,ADFS 2.0,ADHD Coaching,ADI,ADIC,ADINA,ADK,ADL,ADM,ADME,ADMET,ADMS,ADO.NET,ADODB,ADP E-Time,ADP HRB,ADP Payforce,ADP Payroll,ADP ReportSmith,ADP ezLaborManager,ADR Recording,ADRs,ADS-B,ADSI,ADSO,ADT,ADUC,ADX,AE,AEA,AEC,AERMOD,AES,AFAA,AFAA Certified Personal Trainer,AFDX,AFE,AFIS,AFLP,AFM,AFS,AFT Fathom,AFTRA,AGI 32,AHA,AHDL,AHLTA,AHP,AHU,AHWD,AIA,AIA Billing,AIA Documents,AIAG,AIC,AICC,AICP,AIDC,AIESEC,AIF,AIML,AIMSUN,AIMSweb,AIN,AIR,AIR for Android Extension,AISC,AIX 5.x,AIX Administration,AJAX,AJAX Frameworks,AJAX Toolkit,AKTA,ALARP,ALC,ALCO,ALE,ALF,ALLL,ALM,ALM/TFS,ALP,ALSA,ALTA,ALTA Surveys,ALV,ALV Reporting,ALV Reports,AMA style,AMAG,AMB,AMBA,AMBA AHB,AMC,AMCs,AMD64,AMEL,AMESim,AMFI,AMFI Certified,AMFPHP,AMHS,AMI,AML,AMLS,AMOS,AMPL,AMPS,AMQP,AMR,AMRT,AMS360,AMSI Property Management,AMX,AMX Programmer,ANC

In [16]:
pandas_gbq.to_gbq(skills_rec_df, 'Projects.skills_matrix', project_id = project_id, if_exists = "replace")

--------------------------------------------------------------------------- 
 GenericGBQException Traceback (most recent call last)
 <command-1062020953624457> in <module> () 
 ----> 1 pandas_gbq . to_gbq ( skills_rec_df , 'Projects.skills_matrix' , project_id = project_id , if_exists = "replace" ) 

 /databricks/python/lib/python3.5/site-packages/pandas_gbq/gbq.py in to_gbq (dataframe, destination_table, project_id, chunksize, reauth, if_exists, auth_local_webserver, table_schema, location, progress_bar, credentials, verbose, private_key) 
 1144 )
 1145 else : 
 -> 1146 table . create ( table_id , table_schema ) 
 1147 
 1148 if dataframe . empty : 

 /databricks/python/lib/python3.5/site-packages/pandas_gbq/gbq.py in create (self, table_id, schema) 
 1284 self . client . create_table ( table ) 
 1285 except self . http_error as ex : 
 -> 1286 self . process_http_error ( ex ) 
 1287 
 1288 def delete ( self , table_id ) : 

 /databricks/python/lib/python3.5/site-packages/pandas_gbq/gbq.py in process_http_error (ex) 
 402 # <https://cloud.google.com/bigquery/troubleshooting-errors>`__ 
 403 
 --> 404 raise GenericGBQException ( "Reason: {0}" . format ( ex ) ) 
 405 
 406 def run_query ( self , query , ** kwargs ) : 

 GenericGBQException : Reason: 400 POST https://www.googleapis.com/bigquery/v2/projects/deep-rigging-245523/datasets/Projects/tables: Too many total leaf fields: 36944

In [17]:
def skill_recommender(skill_list, matrix):
    skill_recs_all = pd.Series()
    skill_vals_all = pd.Series()  
    for skill in skill_list:
        result = matrix.sort_values(skill,ascending=False)
        skill_recs = pd.Series()
        skill_vals = pd.Series()
        for i in range(1,5): 
            temp_skill_recs = result[skill].index.values[i]
            temp_skill_val = str(round(result[skill][i],3))
            skill_recs = skill_recs.append(pd.Series(temp_skill_recs))
            skill_vals = skill_vals.append(pd.Series(temp_skill_val))
        skill_recs_all = skill_recs_all.append(skill_recs)
        skill_vals_all = skill_vals_all.append(skill_vals)
    skill_recommendations = pd.concat([skill_recs_all, skill_vals_all], axis=1, ignore_index=True, credentials=credentials)
    return skill_recommendations.drop_duplicates(subset='skill_recs_all')

In [18]:
skill_recommender(input_skills, skills_rec_df)